## 1. Spin up cluster

In [20]:
%%bash
aws emr create-cluster \
    --applications Name=Hadoop Name=Spark \
    --ec2-attributes '{"KeyName":"nick_key_pair","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"subnet-0d43037a","EmrManagedSlaveSecurityGroup":"sg-d7bd4baf","EmrManagedMasterSecurityGroup":"sg-d5bd4bad"}'
    --service-role EMR_DefaultRole \
    --enable-debugging \
    --release-label emr-4.6.0 \
    --log-uri 's3n://hamlin-mids-261/logs/' \
    --name 'My cluster' \
    --instance-groups '[{"InstanceCount":1,"BidPrice":"0.05","InstanceGroupType":"MASTER","InstanceType":"m3.xlarge","Name":"Master instance group - 1"}]' \
    --region us-east-1

usage: aws [options] <command> <subcommand> [parameters]
aws: error: argument --ami-version is required
bash: line 4: --service-role: command not found


## 2. Make sure that your EC2 security group allows SSH on port 22 on EMR master node

## 3. SSH to master node and install stuff

In [57]:
#Fill these in
path_to_key_pair='./nick_key_pair.pem'
cluster_public_dns='ec2-52-201-246-11.compute-1.amazonaws.com'

In [58]:
#Connect to master node
print "ssh -i {0} hadoop@{1}".format(path_to_key_pair,cluster_public_dns)

ssh -i ./nick_key_pair.pem hadoop@ec2-52-201-246-11.compute-1.amazonaws.com


In [53]:
#install stuff
print "sudo pip install ipython jupyter"

print ""
#Start the notebook
print 'PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS="notebook --no-browser --port=7777" pyspark'

sudo pip install ipython jupyter

PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS="notebook --no-browser --port=7777" pyspark


## 4. Forward the port

In [59]:
# Forward the local port
# ssh -i .ssh/EMR.pem -N -f -L localhost:7776:localhost:7777 hadoop@public-dns
print "ssh -i {0} -N -f -L localhost:7776:localhost:7777 hadoop@{1}".format(path_to_key_pair,cluster_public_dns)

ssh -i ./nick_key_pair.pem -N -f -L localhost:7776:localhost:7777 hadoop@ec2-52-201-246-11.compute-1.amazonaws.com


## 5. Open http://localhost:7776/ in a local browser to access the notebook

## 6. Move local files back and forth

In [ ]:
# FROM CLUSTER TO LOCAL
# scp your_username@remotehost.edu:foobar.txt /your/local/directory
#scp -i ~/.ssh/EMR.pem hadoop@ec2-54-187-11-233.us-west-2.compute.amazonaws.com: /home/hadoop/a13.ipynb a13.ipynb

# FROM LOCAL TO CLUSTER
# scp foobar.txt your_username@remotehost.edu:/some/remote/directory
#scp -i ~/.ssh/EMR.pem hadoop@ec2-54-187-11-233.us-west-2.compute.amazonaws.com: a13.ipynb /home/hadoop/a13.ipynb

In [55]:
#filename="MIDS-W261-2015-HWK-Week13-Hamlin-Criteo.ipynb"
filename="test.ipynb"

In [60]:
print "FROM CLUSTER TO LOCAL"
print "scp -i {0} hadoop@{1}:/home/hadoop/{2} ./{2} ".format(path_to_key_pair,cluster_public_dns,filename)
print ""
print "FROM LOCAL TO CLUSTER"
print "scp -i {0} ./{2} hadoop@{1}:/home/hadoop/{2}".format(path_to_key_pair,cluster_public_dns,filename)
print ""

FROM CLUSTER TO LOCAL
scp -i ./nick_key_pair.pem hadoop@ec2-52-201-246-11.compute-1.amazonaws.com:/home/hadoop/test.ipynb ./test.ipynb 

FROM LOCAL TO CLUSTER
scp -i ./nick_key_pair.pem ./test.ipynb hadoop@ec2-52-201-246-11.compute-1.amazonaws.com:/home/hadoop/test.ipynb



## Useful References

http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-connect-master-node.html
http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-network-security.html#default-security-group
http://blogs.aws.amazon.com/bigdata/post/TxX4BY5T1PQ7BQ/Using-IPython-Notebook-to-Analyze-Data-with-Amazon-EMR
https://blogs.aws.amazon.com/bigdata/post/Tx15AY5C50K70RV/Installing-Apache-Spark-on-an-Amazon-EMR-Cluster